## Computing lagged returns

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect('db2023.db')

In [3]:
## Adding lagged returns
query = "SELECT * FROM Prices"
df = pd.read_sql_query(query, conn)
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['permno', 'date'])
for i in range(0, 12):
    df[f'ret_lag_{i+1}'] = df.groupby('permno')['retadj'].shift(i)
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df.to_sql('Prices_lags', conn, if_exists='replace', index=False)

4459284

In [4]:
conn.close()

## Processing Characteristics

In [5]:

# Connect to the SQLite database
conn = sqlite3.connect('db2023.db')
cur = conn.cursor()

# Fetch all distinct dates from the Prices table
dates = pd.read_sql_query('SELECT DISTINCT date FROM Prices_lags', conn)
dates_list = dates['date'].tolist()
dates_list.sort()

In [6]:
# Columns to use for ratios
cols_ratios = ['CAPEI', 'bm', 'evm', 'pe_op_basic', 'pe_op_dil', 'pe_exi', 'pe_inc', 'ps', 'pcf',
               'dpr', 'npm', 'opmbd', 'opmad', 'gpm', 'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax',
               'aftret_eq', 'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'pretret_earnat', 'GProf',
               'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio', 'int_debt', 'int_totdebt',
               'cash_lt', 'invt_act', 'rect_act', 'debt_at', 'debt_ebitda', 'short_debt', 'curr_debt',
               'lt_debt', 'profit_lct', 'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be',
               'debt_assets', 'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'cash_ratio',
               'quick_ratio', 'curr_ratio', 'cash_conversion', 'inv_turn', 'at_turn', 'rect_turn',
               'pay_turn', 'sale_invcap', 'sale_equity', 'sale_nwc', 'rd_sale', 'adv_sale', 'staff_sale',
               'accrual', 'ptb', 'PEG_trailing', 'divyield']

# Columns for lagged returns
cols_lags = [f'ret_lag_{i}' for i in range(1, 13)]

# Initialize an empty DataFrame to store results
results = pd.DataFrame()

# Loop through each date
for date in dates_list[:-1]:
    print(f"Processing date: {date}")

    # Load data for the current date
    crsp = pd.read_sql_query('SELECT * FROM Prices_lags WHERE date = ?', conn, params=(date,))
    wifr = pd.read_sql_query('SELECT * FROM Ratios WHERE public_date LIKE ?', conn, params=(date[:7] + '%',))

    # Ensure the columns are numeric
    for column in cols_ratios:
        wifr[column] = pd.to_numeric(wifr[column], errors='coerce')

    # Merge and filter data
    crsp_cols = ["permno", "mktflag", "dlstcd"] + cols_lags
    wifr_mkt = pd.merge(wifr, crsp[crsp_cols], how="inner", on="permno")
    wifr_mkt = wifr_mkt[(wifr_mkt["mktflag"] == 1) & (wifr_mkt["dlstcd"] == "")]

    # Drop the 'mktflag' column
    wifr_mkt.drop(columns=["mktflag"], inplace=True)

    # Convert lagged return columns to numeric
    for col in cols_lags:
        wifr_mkt[col] = pd.to_numeric(wifr_mkt[col], errors='coerce')

    # Compute normalized ranks for the financial ratios and lagged returns
    df_z = wifr_mkt[["gvkey", "permno", "adate", "qdate", "public_date"]].copy()
    cols_ratios_ext = cols_ratios + cols_lags

    for col in cols_ratios_ext:
        rank = wifr_mkt[col].rank(method="average", na_option='keep', ascending=False)
        n_t = rank.count()
        rc = rank / (n_t + 1)
        mu = rc.mean()
        df_z[col] = (rc - mu) / (np.abs(rc - mu).sum())
        df_z[col] = df_z[col].fillna(0)

    # Fetch the next month's date for returns
    next_date = pd.to_datetime(date) + pd.DateOffset(months=1)
    next_date_str = next_date.strftime('%Y-%m-%d')
    rets = pd.read_sql_query('SELECT * FROM Prices WHERE date LIKE ?', conn, params=(next_date_str[:7]+"%",))
    rets = rets[["permno", "retadj"]]
    rets["retadj"] = pd.to_numeric(rets["retadj"], errors='coerce')
    rets.dropna(inplace=True)

    # Merge with z-scores
    df_y = pd.merge(df_z, rets, how="inner", on="permno")
    df_y.set_index("permno", inplace=True)

    # Add date column to df_y
    df_y["date"] = date

    # Append the result to the results DataFrame
    results = pd.concat([results, df_y])
    print(df_y.shape)

# Write the results DataFrame back to the database
results.to_sql('Z_R_matrices', conn, if_exists='replace', index=True)

# Close the database connection
conn.close()

print("Processing completed.")

Processing date: 1970-01-30
(950, 87)
Processing date: 1970-02-27
(946, 87)
Processing date: 1970-03-31
(938, 87)
Processing date: 1970-04-30
(912, 87)
Processing date: 1970-05-29
(892, 87)
Processing date: 1970-06-30
(880, 87)
Processing date: 1970-07-31
(874, 87)
Processing date: 1970-08-31
(884, 87)
Processing date: 1970-09-30
(901, 87)
Processing date: 1970-10-30
(889, 87)
Processing date: 1970-11-30
(888, 87)
Processing date: 1970-12-31
(898, 87)
Processing date: 1971-01-29
(920, 87)
Processing date: 1971-02-26
(930, 87)
Processing date: 1971-03-31
(925, 87)
Processing date: 1971-04-30
(909, 87)
Processing date: 1971-05-28
(916, 87)
Processing date: 1971-06-30
(913, 87)
Processing date: 1971-07-30
(911, 87)
Processing date: 1971-08-31
(919, 87)
Processing date: 1971-09-30
(922, 87)
Processing date: 1971-10-29
(920, 87)
Processing date: 1971-11-30
(912, 87)
Processing date: 1971-12-31
(934, 87)
Processing date: 1972-01-31
(954, 87)
Processing date: 1972-02-29
(970, 87)
Processing d

## Computing excess returns (substract Risk Free Rate)

In [7]:
conn = sqlite3.connect('db2023.db')

In [8]:
query = "SELECT * FROM Z_R_matrices"
results = pd.read_sql_query(query, conn)

In [9]:
# Substracting risk free rate. 
 
# #After this, R_e_adj is the excess return on the table Z_R_matrices.

# Load the Fama_French table
query_ff = "SELECT dateff, rf FROM Fama_French"
fama_french = pd.read_sql_query(query_ff, conn)
fama_french["rf"] = pd.to_numeric(fama_french["rf"], errors='coerce')
# Merge the dataframes on the matching date columns
merged_data = pd.merge(results, fama_french, left_on='date', right_on='dateff')

# Subtract the 'rf' column from the 'retadj' column
merged_data['R_e_adj'] = merged_data['retadj'] - merged_data['rf']

# Drop the 'dateff' and 'rf' columns as they are no longer needed
merged_data.drop(columns=['dateff', 'rf'], inplace=True)

# Write the modified dataframe back to the SQLite database
merged_data.to_sql('Z_R_matrices', conn, if_exists='replace', index=False)

641232

In [10]:
conn.close()

## Export to CSV

In [11]:
# Connect to the SQLite database
conn = sqlite3.connect('db2023.db')

# Load the table into a pandas DataFrame
df = pd.read_sql_query("SELECT * FROM Z_R_matrices", conn)

# Export the DataFrame to a CSV file
df.to_csv('Z_R_matrices.csv', index=False)

# Close the database connection
conn.close()